In [56]:
# ==========================================
# 各種ライブラリと CSV データの読み込み
# ==========================================
# 自動リロードを有効にする設定
%load_ext autoreload
%autoreload 2

# 必要なライブラリのインストール
%pip install pycountry

# ライブラリのインポート
import numpy as np  # 線形代数
import pandas as pd  # データ処理、CSVファイルのI/O（例：pd.read_csv）
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer

# 独自モジュールのインポート
from modules.normalizer import TextNormalizer, AdditionalNormalizer

# CSVデータを pandas データフレームオブジェクトとして読み込み
df = pd.read_csv("../input/nlp-getting-started/train.csv")
df = df.fillna("")  # 空のカラムを空文字に置換

# 元データをコピーして、特徴量とターゲットに分割
X = df[["text", "keyword", "location"]].copy()
y = df["target"].copy()  # ターゲット変数

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [57]:
# ==========================================
# "text" カラムの前処理
# ==========================================
# X["text"] = X["text"][51:101]  # テスト用: 50~100 だけ抽出

X["text"] = (
    X["text"].apply(TextNormalizer.remove_newlines).apply(TextNormalizer.replace_links)
)

In [48]:
# ==========================================
# "keyword" カラムの前処理
# ==========================================

In [58]:
# ==========================================
# "location" カラムの前処理
# ==========================================
# X["location"] = X["location"][51:101]  # テスト用: 50~100 だけ抽出

X["location"] = (
    X["location"]
    .apply(TextNormalizer.remove_numbers_and_symbols)
    .apply(AdditionalNormalizer.normalize_country_name)
)

In [ ]:
# ==========================================
# モデルの学習と評価
# ==========================================
# ダミー変数化
# dummy_na で NaN を特徴量化、今回は空文字を使用しているので本来は不要
# まとめてダミー変数化すると、カラム名が "location_Japan" のようになるため、結合時に衝突しない
X_keyword_and_location = pd.get_dummies(
    X, columns=["keyword", "location"], dummy_na=True
).drop(columns=["text"])

# "text" カラムのテキストデータをTF-IDFベクトルに変換
vectorizer = TfidfVectorizer(max_features=1000)
X_text_vectorized = vectorizer.fit_transform(X["text"]).toarray()

# TF-IDF 特徴量と結合
# pandas は表形式 (二次元) までしか扱えないため、numpy で変換しなおす
X = np.hstack([X_text_vectorized, X_keyword_and_location.values])

# データを訓練セットとテストセットに分割
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# モデルの訓練
clf = RandomForestClassifier(random_state=42)
# clf.fit(X_train, y_train)
clf.fit(X, y)

# テストセットで予測
# y_pred = clf.predict(X_test)
y_pred = clf.predict(X)

# 精度の計算
# accuracy = accuracy_score(y_test, y_pred)
accuracy = accuracy_score(y, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.99


In [62]:
# ==========================================
# 本番データに対する予測
# ==========================================
# test.csv に対して予測を行う
df_prod = pd.read_csv("../input/nlp-getting-started/test.csv")
df_prod = df_prod.fillna("")
X_prod = df_prod[["text", "keyword", "location"]].copy()

# 前処理
X_prod["text"] = (
    X_prod["text"]
    .apply(TextNormalizer.remove_newlines)
    .apply(TextNormalizer.replace_links)
)
X_prod["location"] = (
    X_prod["location"]
    .apply(TextNormalizer.remove_numbers_and_symbols)
    .apply(AdditionalNormalizer.normalize_country_name)
)

# 特徴量の作成
X_prod_text_vectorized = vectorizer.transform(X_prod["text"]).toarray()
X_prod_keyword_and_location = pd.get_dummies(
    X_prod, columns=["keyword", "location"], dummy_na=True
).drop(columns=["text"])

# 訓練データで使用した keyword と location のダミー変数列名を取得
train_feature_cols = X_keyword_and_location.columns

# 訓練データに存在してテストデータに存在しない列を特定
missing_cols = set(train_feature_cols) - set(X_prod_keyword_and_location.columns)

# 欠落した列をすべて一度に追加 (パフォーマンス警告を回避)
if missing_cols:
    missing_df = pd.DataFrame(
        0, index=X_prod_keyword_and_location.index, columns=list(missing_cols)
    )
    X_prod_keyword_and_location = pd.concat(
        [X_prod_keyword_and_location, missing_df], axis=1
    )

# テストデータに存在して訓練データに存在しない列を削除
X_prod_keyword_and_location = X_prod_keyword_and_location[train_feature_cols]

# TF-IDF特徴量と結合
X_prod = np.hstack((X_prod_keyword_and_location.values, X_prod_text_vectorized))
y_pred_prod = clf.predict(X_prod)

# 提出用ファイルの作成
submission = pd.DataFrame({"id": df_prod["id"], "target": y_pred_prod})
submission.to_csv("../output/submission.csv", index=False)